In [1]:
# 调用OpenWeather API
# 使用 requestes 获取天气数据
# 获取天气数据失败提示
# 抓取 json数据

import requests
import json

API_KEY = "7226a2c4ce82265fde9c8d32f42258ec"
CITY = "Dublin"
URL = f"http://api.openweathermap.org/data/2.5/weather?q={CITY}&appid={API_KEY}&units=metric"

def get_weather():
    response = requests.get(URL)
    if response.status_code == 200:
        return response.json()
    else:
        print("Error fetching weather data:", response.status_code)
        return None

if __name__ == "__main__":
    weather_data = get_weather()
    if weather_data:
        print(json.dumps(weather_data, indent=4))


{
    "coord": {
        "lon": -121.9358,
        "lat": 37.7021
    },
    "weather": [
        {
            "id": 701,
            "main": "Mist",
            "description": "mist",
            "icon": "50n"
        }
    ],
    "base": "stations",
    "main": {
        "temp": 7.84,
        "feels_like": 7.84,
        "temp_min": 5.26,
        "temp_max": 8.8,
        "pressure": 1020,
        "humidity": 92,
        "sea_level": 1020,
        "grnd_level": 994
    },
    "visibility": 8047,
    "wind": {
        "speed": 0,
        "deg": 0
    },
    "clouds": {
        "all": 20
    },
    "dt": 1739798889,
    "sys": {
        "type": 1,
        "id": 4774,
        "country": "US",
        "sunrise": 1739804082,
        "sunset": 1739843342
    },
    "timezone": -28800,
    "id": 5344157,
    "name": "Dublin",
    "cod": 200
}


In [2]:
# 创建 MySQL 数据库和表

import pymysql
from sqlalchemy import create_engine, text
from urllib.parse import quote_plus

# 数据库连接信息
USER = "root"
PASSWORD = "Chennan520!"
PORT = "3306"
DB = "current_weather"
URI = "127.0.0.1"

# 创建数据库连接
encoded_password = quote_plus(PASSWORD)
connection_string = f"mysql+pymysql://{USER}:{PASSWORD}@{URI}:{PORT}"
engine = create_engine(connection_string, echo=True)

# 创建数据库（如果不存在）
with engine.connect() as conn:
    conn.execute(text(f"CREATE DATABASE IF NOT EXISTS {DB}"))
    conn.commit()  # 确保变更生效

# 连接到新创建的数据库
connection_string = f"mysql+pymysql://{USER}:{PASSWORD}@{URI}:{PORT}/{DB}"
engine = create_engine(connection_string, echo=True)

# 创建 weather 表
sql = """
CREATE TABLE IF NOT EXISTS weather (
    id INT AUTO_INCREMENT PRIMARY KEY,
    city VARCHAR(50),
    temperature FLOAT,
    feels_like FLOAT,
    humidity INT,
    wind_speed FLOAT,
    timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
"""

# 执行 SQL 语句
with engine.connect() as conn:
    conn.execute(text(sql))
    conn.commit()

print("Database and table setup complete!")


2025-02-17 13:35:51,126 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2025-02-17 13:35:51,127 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-02-17 13:35:51,128 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2025-02-17 13:35:51,129 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-02-17 13:35:51,129 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2025-02-17 13:35:51,129 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-02-17 13:35:51,130 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-02-17 13:35:51,130 INFO sqlalchemy.engine.Engine CREATE DATABASE IF NOT EXISTS current_weather
2025-02-17 13:35:51,130 INFO sqlalchemy.engine.Engine [generated in 0.00045s] {}
2025-02-17 13:35:51,132 INFO sqlalchemy.engine.Engine COMMIT
2025-02-17 13:35:51,134 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2025-02-17 13:35:51,134 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-02-17 13:35:51,135 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2025-02-17 13:35:51,135 INFO sqlalchemy.engin

In [3]:
# 储存天气数据到MySQL, 把API 获取的数据存入数据库

import pymysql
from fetch_weather import get_weather

# 数据库连接信息
USER = "root"
PASSWORD = "Chennan520!"
PORT = "3306"
DB = "current_weather"
URI = "127.0.0.1"

# 连接数据库
def store_weather():
    weather_data = get_weather()
    if not weather_data:
        return

    # 解析需要的数据
    city = weather_data["name"]
    temperature = weather_data["main"]["temp"]
    feels_like = weather_data["main"]["feels_like"]
    humidity = weather_data["main"]["humidity"]
    wind_speed = weather_data["wind"]["speed"]

    # 插入数据库
    conn = pymysql.connect(host=URI, user=USER, password=PASSWORD, database=DB)
    cursor = conn.cursor()
    sql = "INSERT INTO weather (city, temperature, feels_like, humidity, wind_speed) VALUES (%s, %s, %s, %s, %s)"
    cursor.execute(sql, (city, temperature, feels_like, humidity, wind_speed))

    conn.commit()
    conn.close()
    print("Weather data stored successfully!")

if __name__ == "__main__":
    store_weather()


Weather data stored successfully!


In [4]:
# current weather download - json

import requests
import json
import pymysql
import time
import datetime
import os
from pymysql import Error

# 改用 current weather API 而不是 forecast API
API_KEY = "7226a2c4ce82265fde9c8d32f42258ec"
CITY = "Dublin"
URL = f"http://api.openweathermap.org/data/2.5/weather?q={CITY}&appid={API_KEY}&units=metric"

def connect_db():
    try:
        connection = pymysql.connect(
            host="127.0.0.1",
            user="root",
            password="Chennan520!",
            database="current_weather",
            charset="utf8mb4"
        )
        return connection
    except Error as e:
        print(f"Error connecting to MySQL: {e}")
        return None

def extract_current_weather(json_text):
    try:
        data = json.loads(json_text)
        return {
            "timestamp": datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            "temperature": data["main"]["temp"],
            "feels_like": data["main"]["feels_like"],
            "humidity": data["main"]["humidity"],
            "wind_speed": data["wind"]["speed"]
        }
    except Exception as e:
        print(f"Error extracting data: {e}")
        return None

def write_to_db(data):
    if not data:
        print("No data to write to database")
        return False

    conn = None
    cursor = None
    try:
        conn = connect_db()
        if not conn:
            return False

        cursor = conn.cursor()

        sql = """INSERT INTO weather 
                 (timestamp, city, temperature, feels_like, humidity, wind_speed)
                 VALUES (%s, %s, %s, %s, %s, %s)"""
        
        values = (
            data["timestamp"],
            CITY,
            data["temperature"],
            data["feels_like"],
            data["humidity"],
            data["wind_speed"]
        )
        
        cursor.execute(sql, values)
        conn.commit()
        print(f"Successfully inserted weather data for {data['timestamp']}")
        return True

    except Error as e:
        print(f"Database error: {e}")
        if conn:
            conn.rollback()
        return False
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()

def main():
    if not os.path.exists("data"):
        os.mkdir("data")

    while True:
        try:
            current_time = datetime.datetime.now()
            print(f"\nFetching current weather data at {current_time}")
            
            response = requests.get(URL)
            if response.status_code == 200:
                json_data = response.text

                # 保存 JSON 到文件
                filename = f"data/weather_{current_time.strftime('%Y-%m-%d_%H-%M-%S')}.json"
                with open(filename, "w") as f:
                    f.write(json_data)
                print(f"Weather data saved to {filename}")

                # 解析并存入数据库
                weather_data = extract_current_weather(json_data)
                if weather_data:
                    success = write_to_db(weather_data)
                    if success:
                        print("Data successfully written to database")
                    else:
                        print("Failed to write data to database")
                else:
                    print("No data extracted from API response")
            else:
                print(f"Error fetching weather data: {response.status_code}")

            # 休眠10分钟
            print(f"Sleeping for a hour until next update...")
            time.sleep(3600)  # 一小时 = 3600秒

        except Exception as e:
            print(f"Error in main loop: {e}")
            time.sleep(60)  # 发生错误时等待1分钟后重试

if __name__ == "__main__":
    main()


Fetching current weather data at 2025-02-17 13:35:51.230938
Weather data saved to data/weather_2025-02-17_13-35-51.json
Successfully inserted weather data for 2025-02-17 13:35:51
Data successfully written to database
Sleeping for a hour until next update...


KeyboardInterrupt: 